# Different Model Prompts

# LLMs(Current)

## GPT family and Claude family and Mistral Model

**Prompts with Langchain:The langchain use in different models are similar, the difference are initaliztion of chat model and combine them to invoke model to generate output.**




### Initlization and Invoke

***Initlialize the chat model***

In [ ]:
# Initialize the chat model

"""
GPT Faimily
model_name="gpt-4o-mini"/"gpt-4-turbo"/"gpt-3.5-turbo"

"""

llm = OpenAIChat(model_name='The Model Name With Its Version',temperature=0,max_tokens=50)


""""
Claude Family
"""

boto3_bedrock = boto3.client('bedrock-runtime')

# Define the model ID and parameters
model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
#model_id = "us.anthropic.claude-3-5-sonnet-20240620-v1:0"
model_kwargs = {
    "max_tokens": 50,  # Claude-3 use “max_tokens” However Claude-2 requires “max_tokens_to_sample”.
    "temperature": 0.2,
    "top_k": 250,
    "top_p": 0.999,
    #"stop_sequences": ["\n\nHuman"],
    "stop_sequences":[],
}

# Initialize the BedrockChat instance
claude3 = ChatBedrock(
    client=boto3_bedrock,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

"""
Mistral Model
"""

boto3_bedrock = boto3.client('bedrock-runtime', 'us-east-1')

# Define the model ID and parameters
model_id = "mistral.mistral-7b-instruct-v0:2"
model_kwargs = {
    "max_tokens": 50,  # Adjust as needed based on the model's requirements
    "temperature": 0.2
}

# Initialize the BedrockChat instance
mistral = BedrockChat(
    client=boto3_bedrock,
    model_id=model_id,
    model_kwargs=model_kwargs,
)


***Embed the prompt to invoke model and parse output***

In [ ]:
"""GPT Family"""

prompt_value = prompt.format(user_input="The Pinyin Input")
llm_output = llm(prompt_value)

# Parse the output
response = output_parser.parse(llm_output)
print(response.get("ime_string"))

"""Claude Family"""
# Combine the prompt, model, and output parser into a chain
chain = prompt | claude3 | output_parser
response = chain.invoke({"user_input": "The Pinyin Input"})
print(response.get("ime_string"))

""" Mistral Model"""

chain = prompt | mistral | output_parser
response = chain.invoke({"user_input": "The Pinyin Input"})


###### Examples of combining prompt and instructions

Following the complete example using gpt4o-mini and claude3.5 Haiku, same with other gpt claude models

In [ ]:
'''GPT example'''

# Initialize the chat model
llm = OpenAIChat(model_name="gpt-4o-mini",temperature=0,max_tokens=50)


# Define the response schemas
response_schemas = [
    ResponseSchema(name="original_string", description="This an user input Pinyin string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Chinese response")
]
# Initialize the output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get the format instructions
format_instructions = output_parser.get_format_instructions()


# Define the ***Prompt Template***
template = """
You will be given a Pinyin with tone from a user, try to output the corresponding Chinese.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# Leverage the prompt template
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

prompt_value = prompt.format(user_input="jin tian shi ge hao ri zi")
llm_output = llm(prompt_value)

In [ ]:
"""Claude3.5 Haiku Example"""
boto3_bedrock = boto3.client('bedrock-runtime')

# Define the model ID and parameters
model_id = "us.anthropic.claude-3-5-haiku-20241022-v1:0"
model_kwargs = {
    "max_tokens": 50,  # Claude-3 use “max_tokens” However Claude-2 requires “max_tokens_to_sample”.
    "temperature": 0.2,
    "top_k": 250,
    "top_p": 0.999,
    "stop_sequences":[],
}

# Initialize the BedrockChat instance
claude3 = ChatBedrock(
    client=boto3_bedrock,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Define the response schemas
response_schemas = [
    ResponseSchema(name="original_string", description="This is a user input Pinyin string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Chinese response")
]

# Initialize the output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get the format instructions
format_instructions = output_parser.get_format_instructions()

# Define the prompt template
template = """
You will be given a Pinyin with tone from a user, try to output the corresponding Chinese.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# Initialize the prompt template
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

# Combine the prompt, model, and output parser into a chain
chain = prompt | claude3 | output_parser


# Chain Invoke
response = chain.invoke({"user_input": "jin tian shi ge hao ri zi"})
print(response.get("ime_string"))

In [ ]:
"""Mistral Example"""
# Create a Bedrock Runtime client in the AWS Region you want to use.
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")

# Set the model ID, e.g., Mistral.
model_id = "mistral.mistral-7b-instruct-v0:2"
model_kwargs = {
    "max_tokens": 50,
    "temperature": 0.2,
}

# LangChain class for chat
model = ChatBedrock(
    client=bedrock_runtime,
    model_id=model_id,
    model_kwargs=model_kwargs,
)

# Define the response schemas
response_schemas = [
    ResponseSchema(name="original_string", description="This an user input Pinyin sentence"),
    ResponseSchema(name="corrected_string", description="This is your first response, the Chinese response")
    ]
# Initialize the output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get the format instructions
format_instructions = output_parser.get_format_instructions()

# Define the prompt template
messages = [
    ("system", """
    You will be given pinyin strings from a user. Try to output the corresponding Chinese.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | mistral | output_parser

# Invoke the chain

response = chain.invoke({"user_input": "jie guo, qi zhong de 33 feng xin mei le xia ruo.", "format_instructions": format_instructions})

# Print the corresponding Chinese characters from "corrected_string"
corrected_string = response.get("corrected_string", "No corrected_string found")
print(corrected_string)


### Text Generation

##### Prompt  for Chinese Words

df="Chinese_Words"


In [ ]:
## Define the response schemas
response_schemas = [
    ResponseSchema(name="original_string", description="This is a user input Pinyin string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Chinese response")
]
# Initialize the output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
# Get the format instructions
format_instructions = output_parser.get_format_instructions()

In [ ]:

### Define thePrompt Template for GPT and Claude


template = """
You will be given a Pinyin with tone from a user, try to output the corresponding Chinese.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""


## Leverage the prompt template
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

In [ ]:


### Define the ***Prompt Template for Mistral


messages = [
    ("system", """
    You will be given pinyin strings from a user. Try to output the corresponding Chinese.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)

##### Prompts for Chinese Sentences

**Simplified Chinese**

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This a formatted user input pinyin string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Simplified Chinese sentences response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()


"""GPT and Claude"""

template = """
You will be given a formatted pinyin strings from a user.
Printout its Simplified Chinese sentences by the given string.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""


In [ ]:
"""Mistral"""
### Define the ***Prompt Template for Mistral

messages = [
    ("system", """
    You will be given pinyin strings from a user. Try to its Simplified Chinese sentences.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)
# Invoke the chain

response = chain.invoke({"user_input": "The Pinyin Input", "format_instructions": format_instructions})

# Print the corresponding Chinese characters from "corrected_string"
print(response.get("ime_string", "No ime_string found"))

**Traditional Chinese**

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This a formatted user input pinyin string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Traditional Chinese sentences response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

template = """
You will be given a formatted pinyin strings from a user.
Printout its Traditional Chinese sentences by the given string.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""





# Embed our format description into prompt and tell llm what format we want him to output
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)
prompt_value = prompt.format(user_input="The Pinyin Input")
llm_output = llm(prompt_value)
# Parse the output
parsed_output = output_parser.parse(llm_output).get("ime_string")
print(parsed_output)

In [ ]:
"""Mistral"""
### Define the ***Prompt Template for Mistral

messages = [
    ("system", """
    You will be given pinyin strings from a user. Try to its Traditional Chinese sentences.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | mistral | output_parser



# Invoke the chain

response = chain.invoke({"user_input": "The Pinyin Input", "format_instructions": format_instructions})

# Print the corresponding Chinese characters from "ime_string"
response.get("ime_string", "No ime_string found")


##### Prompt for Japanese Words


df="Japanese_Words"

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This a formatted user input romanji string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Japanese response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

**Kanji**

In [ ]:
template_kanji = """
You will be given a formatted romanji string from a user.
Printout its Kanji by the given string.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# Embed our format description into prompt and tell llm what format we want him to output
prompt_kanji= PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template_kanji
)

**Hiragana**

In [ ]:
template_hira = """
You will be given a formatted romanji string from a user.
Printout its Hiragana by the given string.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# Embed our format description into prompt and tell llm what format we want him to output
prompt_hira = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template_hira
)

**Katakana**

In [ ]:
template_kata = """
You will be given a formatted romanji string from a user.
Printout its katakana by the given string.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# Embed our format description into prompt and tell llm what format we want him to output
prompt_kata = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template_kata
)

In [ ]:
# if Kanji, prompt=prompt_kanji
# if Hiragana,prompt=prompt_hira
# if Katakana, prompt=prompt_kata

promptValue = prompt.format(user_input= 'The Romanji Input')
llm_output = llm(promptValue)
t1_stop = time.time()
ime_string=output_parser.parse(llm_output).get("ime_string")

##### Prompt for Japanese Sentences


df="Japanese_Sentences"

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This a formatted user input romanji string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Japanese response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()

template = """
You will be given a formatted romanji string from a user.
Printout its Japaneses by the given string.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# Embed our format description into prompt and tell llm what format we want him to output
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template)

In [ ]:
"""Mistral Model"""
### Define the ***Prompt Template for Mistral

messages = [
    ("system", """
    Please convert the provided Romanji input into its most accurate corresponding Japanese characters.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)


### Textual Error Correction



In [ ]:

model_name="gpt-4o"/"gpt4-turbo"/"gpt3.5-turbo"

llm = OpenAIChat(model_name="gpt-4o",temperature=0)


#### Chinese

##### Pinyin-based Error Prompt

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This an incorrect user input Pinyin sentence"),
    ResponseSchema(name="corrected_string", description="This is your first response, the correct Chinese response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [ ]:
# Prompt for Template
template = """
You will be given strings with incorrect Pinyin from a user,  which may contain
some numbers, various kinds of typos including missing and adding some letters,
mispress nearby letter keys,  and etc.

Typo Examples:

Input: "ni hai ma?"
Error: "hai" should be "hao"
Corrected Pinyin: "ni hao ma?"
Output Chinese: "你好吗?"

Input: "zhee shi wo de sh."
Error: "zhee" should be "zhe"; "sh" should "shu"
Corrected Pinyin: "zhe shi wo de shu."
Output Chinese: "这是我的书."

Input: "ta s wo de peng you."
Error: "s" should be "shi"
Corrected Pinyin: "ta shi wo de pengyou."
Output Chinese: "他是我的朋友."

Input: "qianmian yu yi jia diai."
Error: "yu" should be "you";"diai" should be "dian"
Corrected Pinyin: "qianmian you yi jia dian."
Output Chinese: "前面有一家店."

Try to correct the Pinyin and output the corresponding Chinese.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""



# Embed our format description into prompt and tell llm what format we want him to output
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)
promptValue = prompt.format(user_input= "noise Pinyin input")
llm_output = llm (promptValue)
output_parser.parse(llm_output).get("corrected_string")

In [ ]:
"""Mistral Model"""

messages = [
    ("system", """
      You will be given strings with incorrect Pinyin from a user,  which may contain
      some numbers, various kinds of typos including missing and adding some letters,
      mispress nearby letter keys,  and etc.

      Typo Examples:

      Input: "ni hai ma?"
      Error: "hai" should be "hao"
      Corrected Pinyin: "ni hao ma?"
      Output Chinese: "你好吗?"

      Input: "zhee shi wo de sh."
      Error: "zhee" should be "zhe"; "sh" should "shu"
      Corrected Pinyin: "zhe shi wo de shu."
      Output Chinese: "这是我的书."

      Input: "ta s wo de peng you."
      Error: "s" should be "shi"
      Corrected Pinyin: "ta shi wo de pengyou."
      Output Chinese: "他是我的朋友."

      Input: "qianmian yu yi jia diai."
      Error: "yu" should be "you";"diai" should be "dian"
      Corrected Pinyin: "qianmian you yi jia dian."
      Output Chinese: "前面有一家店."

      Try to correct the Pinyin and output the corresponding Chinese.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | mistral | output_parser

# Invoke the chain
response = chain.invoke({"user_input": "The Pinyin Input", "format_instructions": format_instructions})

# Print the corresponding Chinese characters from "corrected_string"
response.get("corrected_string", "No corrected_string found")


##### Chinese Character-based Error Prompt

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This an incorrect user input Chinese sentence"),
    ResponseSchema(name="corrected_string", description="This is your first response, the correct Chinese response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

format_instructions = output_parser.get_format_instructions()


In [ ]:
"""GPT and Claude Family"""

# Prompt for Template
template = """
You will be given sentences with incorrect Chinese characters from a user,  which
may contain some numbers, various kinds of errors including wrong characters with
typos, word misuse, grammatical errors, and more.

Error Examples:

Input: "白血斌"
Error: "斌" should be "病"
Corrected Chinese: "白血病"

Input: "我要去北京tiananman广场玩儿。"
Error: "tiananman" should be "天安门"
Corrected Chinese: "我要去北京天安门广场玩儿。"

Input: "金秋9月，给大高校开学的钟声陆续敲响，全国各地先去了周边短租热潮"
Error: "给大" should be "各大"; "先去" should "掀起"
Corrected Chinese: "金秋9月，各大高校开学的钟声陆续敲响，全国各地掀起了周边短租热潮"


Input: "意味着这些第3方应用灭下来将无法向用户发送私"
Error: "灭下来" should be “接下来"; "私" should be "私信"
Corrected Chinese: "意味着这些第3方应用接下来将无法向用户发送私信"


Try to correct the wrong Chinese characters and output the corresponding correct Chinese sentence.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""





# Embed our format description into prompt and tell llm what format we want him to output
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)
promptValue = prompt.format(user_input= "noise Chinese character input")
llm_output = llm (promptValue)
output_parser.parse(llm_output).get("corrected_string")


In [ ]:
"""Mistral Model"""

messages = [
    ("system", """
      You will be given sentences with incorrect Chinese characters from a user,  which may contain some numbers, various kinds of
      errors including wrong characters with typos, word misuse, grammatical errors, and more.

      Error Examples:

      Input: "白血斌"
      Error: "斌" should be "病"
      Corrected Chinese: "白血病"

      Input: "我要去北京tiananman广场玩儿。"
      Error: "tiananman" should be "天安门"
      Corrected Chinese: "我要去北京天安门广场玩儿。"

      Input: "金秋9月，给大高校开学的钟声陆续敲响，全国各地先去了周边短租热潮"
      Error: "给大" should be "各大"; "先去" should "掀起"
      Corrected Chinese: "金秋9月，各大高校开学的钟声陆续敲响，全国各地掀起了周边短租热潮"


      Input: "意味着这些第3方应用灭下来将无法向用户发送私"
      Error: "灭下来" should be “接下来"; "私" should be "私信"
      Corrected Chinese: "意味着这些第3方应用接下来将无法向用户发送私信"

    Try to correct the wrong Chinese characters and output the corresponding correct Chinese sentence.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]



# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | mistral | output_parser

# Invoke the chain
response = chain.invoke({"user_input": "The Pinyin Input", "format_instructions": format_instructions})
# Print the corresponding Chinese characters from "corrected_string"
response.get("corrected_string", "No corrected_string found")


#### Japanese

##### Romanji-based Error Prompt

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This an incorrect user input romanji sentence"),
    ResponseSchema(name="corrected_string", description="This is your first response, the correct Japanese response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [ ]:
""" GPT and Claude Family"""

template = """
You will be given strings with incorrect Romanji from a user,  which may contain
various kinds of typos including missing or adding some letters, mispress or
substitute other characters, and grammatical errors and more.

Typo Examples:

Input: "Ani ni butia shozoku"
Error: "no" should be "ni";"butia" should be "butai"
Corrected Romanji: "Ani no butai shozoku"
Output Japanese: "兄の部隊所属"

Input: "amerika deha 20 seiki nakaba made hakujin no kayou gakkou to kokujin no kayou gakkou ni waka reita."
Error: "reita" should be "reteita"
Corrected Romanji: "amerika deha 20 seiki nakaba made hakujin no kayou gakkou to kokujin no kayou gakkou ni waka reteita."
Output Japanese: "アメリカでは20世紀半ばまで白人の通う学校と黒人の通う学校に分かれいた."

Input: "shikashii nyuujiirandokannpanii to imin ha uragiri dato kanji ta."
Error: "shikashii" should be "shikashi";"nyuujiirandokannpanii" should be "nyuujiirandokanpanii"
Corrected Romanji: "shikashi nyuujiirandokanpanii to imin ha uragiri dato kanji ta."
Output Japanese: "しかしニュージーランドカンパニーと移民は裏切りだと感じた."



Try to correct the Romanji and output the corresponding Japaneses.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# Embed our format description into prompt and tell llm what format we want him to output
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)
promptValue = prompt.format(user_input= "noise Romanji input")
llm_output = llm (promptValue)
output_parser.parse(llm_output).get("corrected_string")

In [ ]:
""" Mistral Model"""

messages = [
    ("system", """
    You will be given strings with incorrect Romanji from a user,  which may contain
    various kinds of typos including missing or adding some letters, mispress or
    substitute other characters, and grammatical errors and more.

    Typo Examples:

    Input: "Ani ni butia shozoku"
    Error: "no" should be "ni";"butia" should be "butai"
    Corrected Romanji: "Ani no butai shozoku"
    Output Japanese: "兄の部隊所属"

    Input: "amerika deha 20 seiki nakaba made hakujin no kayou gakkou to kokujin no kayou gakkou ni waka reita."
    Error: "reita" should be "reteita"
    Corrected Romanji: "amerika deha 20 seiki nakaba made hakujin no kayou gakkou to kokujin no kayou gakkou ni waka reteita."
    Output Japanese: "アメリカでは20世紀半ばまで白人の通う学校と黒人の通う学校に分かれいた."

    Input: "shikashii nyuujiirandokannpanii to imin ha uragiri dato kanji ta."
    Error: "shikashii" should be "shikashi";"nyuujiirandokannpanii" should be "nyuujiirandokanpanii"
    Corrected Romanji: "shikashi nyuujiirandokanpanii to imin ha uragiri dato kanji ta."
    Output Japanese: "しかしニュージーランドカンパニーと移民は裏切りだと感じた."

    Try to correct the Romanji and output the corresponding Japaneses.

    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | model | output_parser
# Invoke the chain
response = chain.invoke({"user_input": "The Pinyin Input", "format_instructions": format_instructions})

# Print the corresponding Chinese characters from "corrected_string"
response.get("corrected_string", "No corrected_string found")


##### Japanese Character-based Error Prompt

In [ ]:
response_schemas = [
    ResponseSchema(name="original_string", description="This an incorrect user input Japanese sentence"),
    ResponseSchema(name="corrected_string", description="This is your first response, the correct Japanese response")
]

#Initialization
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()


In [ ]:
""" GPT and Claude Family"""

# Prompt for Template
template = """
You will be given sentences with incorrect Japanese characters from a user, which
may contain various kinds of errors including replacement of a character with an
erroneous one,the omission of a necessary character, the addition of an
unnecessary character, and character misuse, grammatical errors, and more.

Error Examples:

Input: "全学全てが大学院に以降していないため。"
Error: "以降" should be "移行"
Corrected Japanese: "全学全てが大学院に移行していないため。"

Input: "兄弟がいないため、兄弟の多い六助を羨ましいと思ってる節がある。"
Error: "てる" should be "ている"
Corrected Japanese: "兄弟がいないため、兄弟の多い六助を羨ましいと思っている節がある。"

Input: "そして転校初日に校舎裏に呼び出さる。"
Error: "る" should be "れる"
Corrected Japanese: "そして転校初日に校舎裏に呼び出される。"


Try to correct the wrong Japanses characters and output the corresponding correct Japanese sentence.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""


# Embed our format description into prompt and tell llm what format we want him to output
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)
promptValue = prompt.format(user_input= "noise Chinese character input")
llm_output = llm (promptValue)
output_parser.parse(llm_output).get("corrected_string")

In [ ]:
""" Mistral Model"""

messages = [
    ("system", """
      You will be given sentences with incorrect Japanese characters from a user, which
      may contain various kinds of errors including replacement of a character with an
      erroneous one,the omission of a necessary character, the addition of an
      unnecessary character, and character misuse, grammatical errors, and more.

      Error Examples:

      Input: "全学全てが大学院に以降していないため。"
      Error: "以降" should be "移行"
      Corrected Japanese: "全学全てが大学院に移行していないため。"

      Input: "兄弟がいないため、兄弟の多い六助を羨ましいと思ってる節がある。"
      Error: "てる" should be "ている"
      Corrected Japanese: "兄弟がいないため、兄弟の多い六助を羨ましいと思っている節がある。"

      Input: "そして転校初日に校舎裏に呼び出さる。"
      Error: "る" should be "れる"
      Corrected Japanese: "そして転校初日に校舎裏に呼び出される。"


      Try to correct the wrong Japanses characters and output the corresponding correct Japanese sentence.
    """),
    ("human", """
    {format_instructions}

    % USER INPUT:
    {user_input}

    YOUR RESPONSE:
    """)
]

# Configure Prompt and Chain (using LCEL)
prompt = ChatPromptTemplate.from_messages(messages)
chain = prompt | model | output_parser
# Invoke the chain
response = chain.invoke({"user_input": "The Pinyin Input", "format_instructions": format_instructions})

# Print the corresponding Chinese characters from "corrected_string"
response.get("corrected_string", "No corrected_string found")


## Qwen2 Model(Only for Chinese)

***We can use langchain for Qwen2 Model or not use langchain for it. If use langchain, it is similar with GPT,Claude and Mistral Model***

**Reminder： The Prompt Instruction of Qwen2 model supports English and Chinese.**

#### ***langchain***

All other task prompt template is same with above.

In [ ]:
"""Langchain"""
# Define the response schemas
response_schemas = [
    ResponseSchema(name="original_string", description="This is a user input Pinyin string"),
    ResponseSchema(name="ime_string", description="This is your first response, the Chinese response")
]


# Initialize the output parser
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# Get the format instructions
format_instructions = output_parser.get_format_instructions()

# Define the prompt template
template = """
You will be given a Pinyin with tone from a user, try to output the corresponding Chinese.

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""



final_prompt = prompt.format(user_input="The Pinyin Input")
# Define the messages for the chat model
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": final_prompt}
]

# Initialize the tokenizer and model

# Apply the chat template using the tokenizer
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

# Prepare the model inputs
model_inputs = tokenizer([text], return_tensors="pt").to(device)

# Generate the response from the model
generated_ids = model.generate(
    model_inputs.input_ids,
    max_new_tokens=len(input),
    temperature = 0.2
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

# Decode the response
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
# Parse the structured response
parsed_response = output_parser.parse(response)
parsed_response.get("ime_string")

***Following is another template which doesn't use langchain.***

#### No langchain

##### Prompt for Chinese Words

In [ ]:
def qwen2_cw_generation(input, verbose=False):
  #Below is a combination of instructions describing the task and input with context. Given a response that appropriately meets the instructions.

  ### English Version
  """
    try:
      instruction = "Convert the given Pinyin with tones into the most accurate corresponding Chinese characters."
      prompt=f"""

      ### instruction:
      {instruction}

      ### input:
      {input}

      ### response:
      """
  """
  ### Chinese Version
    try:
      instruction = "将给定的带声调拼音转换为最准确对应的中文字符。"
      prompt = f"""

      ### 指令:
      {instruction}

      ### 输入:
      {input}

      ### 回答:
      """


      token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

      with torch.no_grad():
          output_ids = model.generate(
              token_ids.to(model.device),
              max_new_tokens=50,
              do_sample=True,
              temperature=0.2,
              pad_token_id=tokenizer.pad_token_id,
              bos_token_id=tokenizer.bos_token_id,
              eos_token_id=tokenizer.eos_token_id
          )

      output = tokenizer.decode(output_ids.tolist()[0])

      if verbose:
        print(output)

      #return output.split("### response:\n")[1].split("\n")[0].strip()
      return output.split("### 回答:\n")[1].split("\n")[0].strip()
  except:
      return '';

##### Prompt for Chinese Sentences

***Simplified Version***

In [ ]:
def qwen2_cs_generation_simp(input, verbose=False):
  #Below is a combination of instructions describing the task and input with context. Given a response that appropriately meets the instructions.

  ### English Version
  '''
  try:
      instruction = "Convert the given Pinyin into the most accurate corresponding Simplified Chinese characters."
      prompt=f"""

      ### instruction:
      {instruction}

      ### input:
      {input}

      ### response:
      """
  '''
  ## Chinese Version
  try:
      instruction = "将给定的带声调拼音转换为最准确对应的简体中文字符。"
      prompt = f"""

      ### 指令:
      {instruction}

      ### 输入:
      {input}

      ### 回答:
    """



      token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

      with torch.no_grad():
          output_ids = model.generate(
              token_ids.to(model.device),
              max_new_tokens=200,
              do_sample=True,
              pad_token_id=tokenizer.pad_token_id,
              bos_token_id=tokenizer.bos_token_id,
              eos_token_id=tokenizer.eos_token_id
          )

      output = tokenizer.decode(output_ids.tolist()[0])

      if verbose:
        print(output)

      #return output.split("### response:\n")[1].split("\n")[0].strip()
      return output.split("### 回答:\n")[1].split("\n")[0].strip()
  except:
      return '';

***Traditional Version***

In [ ]:
def qwen2_cs_generation_trad(input, verbose=False):
  #Below is a combination of instructions describing the task and input with context. Given a response that appropriately meets the instructions.

  ### English Version
  '''
  try:
      instruction = "Convert the given Pinyin into the most accurate corresponding Traditional Chinese characters."
      prompt=f"""

      ### instruction:
      {instruction}

      ### input:
      {input}

      ### response:
      """
  '''
  ## Chinese Version
  try:
      instruction = "将给定的带声调拼音转换为最准确对应的繁体中文字符。"
      prompt = f"""

      ### 指令:
      {instruction}

      ### 输入:
      {input}

      ### 回答:
    """

      token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

      with torch.no_grad():
          output_ids = model.generate(
              token_ids.to(model.device),
              max_new_tokens=200,
              do_sample=True,
              pad_token_id=tokenizer.pad_token_id,
              bos_token_id=tokenizer.bos_token_id,
              eos_token_id=tokenizer.eos_token_id
          )

      output = tokenizer.decode(output_ids.tolist()[0])

      if verbose:
        print(output)

      #return output.split("### response:\n")[1].split("\n")[0].strip()
      return output.split("### 回答:\n")[1].split("\n")[0].strip()
  except:
      return '';

##### Prompt for Chinese Sentence Correction

###### Incorrect Pinyin

In [ ]:
def qwen2_csc_pinyinn(input, verbose=False):
    try:

      prompt=f"""
      以下输入的拼音字符不正确，其中可能包括缺失或多余的字母、按错的键或放错的数字等拼写错误。

      尝试纠正给定的错误拼音输入，并输出与之对应的最准确的中文。

      ### 输入:
      {input}

      ### 回答:
      """


      token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

      with torch.no_grad():
          output_ids = model.generate(
              token_ids.to(model.device),
              max_new_tokens=len(input),
              do_sample=True,
              temperature=1,
              pad_token_id=tokenizer.pad_token_id,
              bos_token_id=tokenizer.bos_token_id,
              eos_token_id=tokenizer.eos_token_id
          )

      output = tokenizer.decode(output_ids.tolist()[0])

      if verbose:
        print(output)

      return output.split("### 回答:\n")[1].split("\n")[0].strip().strip("<|im_end|>")
   except:
      return '';

#English Version
'''
 try:
      instruction ="Correct the given Pinyin input and output the most accurate corresponding Chinese characters."
      prompt=f"""

      You are provided with incorrect Pinyin, which may include typos
      such as missing or extra letters, mispressed keys, or misplaced numbers.


      ### instruction:
      {instruction}

      ### input:
      {input}

      ### response:
      """

  '''

###### Incorrect Chinese Characters

In [ ]:
def qwen2_csc_incorrectstrings(input, verbose=False):
  try:

      prompt=f"""
      以下输入的中文字符不正确，其中可能包括语法错误，缺失或多余的字母、按错的键或放错的数字等拼写错误。

      尝试纠正输入错误的中文字符。

      ### 输入:
      {input}

      ### 回答:
      """

      token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

      with torch.no_grad():
          output_ids = model.generate(
              token_ids.to(model.device),
              max_new_tokens=1.5*len(input),
              temperature=0.2,
              pad_token_id=tokenizer.pad_token_id,
              bos_token_id=tokenizer.bos_token_id,
              eos_token_id=tokenizer.eos_token_id
          )

      output = tokenizer.decode(output_ids.tolist()[0])

      if verbose:
        print(output)

      return output.split("### 回答:\n")[1].split("\n")[0].strip().strip("<|im_end|>")
  except:
      return '';

#English Version
'''
  try:
      instruction ="Correct the given erroneous Chinese input and output the most accurate corresponding Chinese characters."
      prompt=f"""

      You are provided with incorrect Chinese characters, which may include typos
      such as missing or extra letters, mispressed keys, or misplaced numbers.

      ### instruction:
      {instruction}

      ### input:
      {input}

      ### response:
      """

'''

Kanji

In [ ]:
def rn_jw_kanji_generation(input, verbose=False):
  try:
    instruction = "ローマ字を日本語の漢字に変換してください。可能な限り正確に翻訳してください。"
    prompt = f"""
    以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字入力を日本語の漢字に正確に変換します。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=100,
            do_sample=True,
            temperature=0.2,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])
    output = tokenizer.decode(output_ids.tolist()[0])
    #print(output)
    if verbose:
      print(output)

    return output.replace("<|endoftext|>", "").split("### 応答:\n")[1].split("\n")[0].strip()
  except:
    return '';

In [ ]:
rn_jw_kanji_generation("kareha shigoto notamenisono konsaato he ike nakatta.")
#rn_jw_kanji_generation("akarasama")

'カレハは実家から離れた場所で仕事を始める。'

Hiragana

In [ ]:
def rn_jw_hira_generation(input, verbose=False):
  try:
    instruction = "ローマ字入力を正確にひらがなに変換。可能な限り正確に翻訳してください。"
    prompt = f"""
    以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字入力を正確にひらがなに変換。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=100,
            do_sample=True,
            temperature=0.2,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])

    if verbose:
      print(output)

    return output.replace("<|endoftext|>", "").split("### 応答:\n")[1].split("\n")[0].strip()
  except:
    return '';

Katagana

In [ ]:
def rn_jw_kata_generation(input, verbose=False):
  try:
    instruction = "ローマ字入力を正確にカタカナに変換します。可能な限り正確に翻訳してください。"
    prompt = f"""
    以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字入力を正確にカタカナに変換します。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=100,
            do_sample=True,
            temperature=0.2,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])

    if verbose:
      print(output)

    return output.replace("<|endoftext|>", "").split("### 応答:\n")[1].split("\n")[0].strip()
  except:
    return '';

case example for textual generation of Japanese

In [ ]:
#input = "kareha shigoto notamenisono konsaato he ike nakatta."
def rn_js_generation(input):
  instruction = "ローマ字を日本語に変換"
  prompt = f"""
  以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

  ### 指示:
  {instruction}

  ### 入力:
  {input}

  ### 応答:
  """

  token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

  with torch.no_grad():
      output_ids = model.generate(
          token_ids.to(model.device),
          max_new_tokens=len(input),
          do_sample=True,
          temperature=1,
          pad_token_id=tokenizer.pad_token_id,
          bos_token_id=tokenizer.bos_token_id,
          eos_token_id=tokenizer.eos_token_id
      )

  output = tokenizer.decode(output_ids.tolist()[0])
  return output.split("### 応答:\n")[1].strip("<|endoftext|>")

In [ ]:
text = "西田幾多郎は、"
token_ids = tokenizer.encode(text, add_special_tokens=False, return_tensors="pt")

with torch.no_grad():
    output_ids = model.generate(
        token_ids.to(model.device),
        max_new_tokens=200,
        min_new_tokens=200,
        do_sample=True,
        temperature=1.0,
        top_p=0.95,
        pad_token_id=tokenizer.pad_token_id,
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id
    )

output = tokenizer.decode(output_ids.tolist()[0])
print(output)

西田幾多郎は、自らの「学問上の動機」を次のように語っている。哲学者として、「私は特に実在の問題ということに興味を持っています。それは私にとっては人生の問題そのものであり、世界の問題そのものなのです」。西田幾多郎にとっての実在とは何か。それは「私は実在は本質の問題だと思います。実在は存在そのものというよりも本質そのものの問題だと思います」。実在の問題が、本質の問題である。実在とは、世界の問題ではなく人生の問題である、と西田幾多郎は語っている。西田幾多郎と友永威烈の対談のなかで、西田幾多郎は、実在が人生の問題であるという理由として「私は実在の問題ということを人生と直接に係わっていると思うからです。実在の問題ということは即ちその存在論を意味するものですからそれは私が如何にして


## Rinna and Rakuten Model for Japanese

**Reminder： The Prompt Instruction of Rakuten model supports Japanases and CHinese.** Please use Japanese as the instruction to launch model.

##### Japanese Words

Kanji

In [ ]:
def rn_jw_kanji_generation(input, verbose=False):
  try:
    instruction = "ローマ字を日本語の漢字に変換してください。可能な限り正確に翻訳してください。"
    prompt = f"""
    以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字入力を日本語の漢字に正確に変換します。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=100,
            do_sample=True,
            temperature=0.5,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])
    output = tokenizer.decode(output_ids.tolist()[0])
    #print(output)
    if verbose:
      print(output)

    return output.replace("<|endoftext|>", "").split("### 応答:\n")[1].split("\n")[0].strip()
  except:
    return '';

Hiragana

In [ ]:
def rn_jw_hira_generation(input, verbose=False):
  try:
    instruction = "ローマ字入力を正確にひらがなに変換。可能な限り正確に翻訳してください。"
    prompt = f"""
    以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字入力を正確にひらがなに変換。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=100,
            do_sample=True,
            temperature=0.2,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])

    if verbose:
      print(output)

    return output.replace("<|endoftext|>", "").split("### 応答:\n")[1].split("\n")[0].strip()
  except:
    return '';

Katagana

In [ ]:
def rn_jw_kata_generation(input, verbose=False):
  try:
    instruction = "ローマ字入力を正確にカタカナに変換します。可能な限り正確に翻訳してください。"
    prompt = f"""
    以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字入力を正確にカタカナに変換します。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=100,
            do_sample=True,
            temperature=0.2,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])

    if verbose:
      print(output)

    return output.replace("<|endoftext|>", "").split("### 応答:\n")[1].split("\n")[0].strip()
  except:
    return '';

##### Japanese Sentence

In [ ]:
def rn_js_generation(input, verbose=False):
  try:
    instruction = "ローマ字を日本語の文字に変換してください。可能な限り正確に翻訳してください。"
    prompt = f"""
    以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字の入力を、日本語の文字に正確に変換してください。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """


    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(
            token_ids.to(model.device),
            max_new_tokens=300,
            do_sample=True,
            temperature=0.9,
            pad_token_id=tokenizer.pad_token_id,
            bos_token_id=tokenizer.bos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )

    output = tokenizer.decode(output_ids.tolist()[0])

    if verbose:
      print(output)

    return output.replace("<|endoftext|>", "").split("### 応答:\n")[1].split("\n")[0].strip()
  except:
    return '';

### Japanese Error Correction

#### Incorrect Romanji

In [ ]:
def rn_jsc_generation(input, verbose=False):
    try:
        instruction = "ローマ字の誤りを修正し、正確に日本語の文字に変換してください。"
        prompt = f"""
        以下は、誤ったローマ字を含む入力文字列であり、文字の欠落、追加、置換、印刷ミスや文法的な誤り
        が含まれている可能性があります。ローマ字を修正し、それを正確に日本語の文字に変換してください。

        ### 指示:
        {instruction}

        ### 入力:
        {input}

        ### 応答:
        """

        token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)

        with torch.no_grad():
            output_ids = model.generate(
                token_ids,
                max_new_tokens=500,
                do_sample=True,
                temperature=0.9,  # Adjusted to your previous setting
                pad_token_id=tokenizer.eos_token_id,
                bos_token_id=tokenizer.bos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True).strip()

        if verbose:
            print(output)

        # Extract and return only the response part
        return output.split("### 応答:\n")[1].split("\n")[0].strip()

    except Exception as e:
        if verbose:
            print(f"Error: {e}")
        return ''


#### Incorrect Japanese Characters

In [ ]:
def rn_jsc_char_generation(input, verbose=False):
  try:

    instruction = "不正確な日本語の文字を修正し、正確な日本語の文章を出力してください。"

    prompt = f"""
    以下は、不正確な日本語の文字を含むユーザーの入力文です。これには、文字の欠落、追加、誤字、置換、
    文法的な誤りなどが含まれている可能性があります。誤りを修正し、正しい日本語の文章を出力してください。

    ### 指示:
    {instruction}

    ### 入力:
    {input}

    ### 応答:
    """

    token_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors="pt").to(model.device)

    with torch.no_grad():
          output_ids = model.generate(
              token_ids,
              max_new_tokens=500,
              do_sample=True,
              temperature=0.9,  # Adjusted to your previous setting
              pad_token_id=tokenizer.eos_token_id,
              bos_token_id=tokenizer.bos_token_id,
              eos_token_id=tokenizer.eos_token_id
          )

    output = tokenizer.decode(output_ids.tolist()[0], skip_special_tokens=True).strip()

    if verbose:
        print(output)

      # Extract and return only the response part
    return output.split("### 応答:\n")[1].split("\n")[0].strip()

  except Exception as e:
      if verbose:
          print(f"Error: {e}")
      return ''

## Baseline Model(Current)



For Chinese, *pypinyin* package converts Chinese to Pinyin and *Pinyin2Hanzi* are rare pacakge that can achieve Pinyin to Chinese.

Here we import *Pinyin2Hanzi* as one of baseline models for Chinese Phonetic-based (Pinyin) textual generation task.


### Pinyin2Hanzi for Chinese
(Words and Sentences)

1. HMM Transformation \\
2. DAG Transformation \\


The github of Pinyin2Hanzi is:
https://github.com/letiantian/Pinyin2Hanzi

#### HMM

In [ ]:
import re, Pinyin2Hanzi
from Pinyin2Hanzi import is_pinyin,simplify_pinyin
from Pinyin2Hanzi import DefaultHmmParams,DefaultDagParams,dag
from Pinyin2Hanzi import viterbi


hmmparams = DefaultHmmParams()
pre = re.compile(u'[\u4e00-\u9fa5-\，\。]')

for j in range(df.shape[0]):

    t1_start=time.time()
    hmm = viterbi(hmm_params=hmmparams, observations=(simplify_pinyin(df.pinyin.iloc[j]).split()), path_num = 1)

    for i,item in enumerate(hmm):
      df.P2H_HMM_gen.iloc[j] = ''.join(re.findall(pre, str(item.path)))
    t1_stop=time.time()
    df.P2H_HMM_time.iloc[j] = t1_stop - t1_start
    print("No",j,"HMM_gen",df.P2H_HMM_gen.iloc[j])
    print("No",j,"HMM_time",t1_stop - t1_start)

In [ ]:
'''
df=Chinese Sentence
'''
hmmparams = DefaultHmmParams()
pre = re.compile(u'[\u4e00-\u9fa5-\，\。]')

for j in range(df.shape[0]):
  text=re.sub(r'\d+','',simplify_pinyin(df.pinyin_nopun.iloc[j]))

  t1_start=time.time()
  hmm = viterbi(hmm_params=hmmparams, observations=(text.split()), path_num = 1)
  for i,item in enumerate(hmm):
      df.P2H_HMM_gen.iloc[j] = ''.join(re.findall(pre, str(item.path)))
  t1_stop=time.time()

#### DAG

In [ ]:
'''
df=Chinese word
'''
dagparams = DefaultDagParams()
pre = re.compile(u'[\u4e00-\u9fa5-\，\。]')

for j in range(df.shape[0]):
    t1_start=time.time()
    dagg = dag(dagparams, simplify_pinyin(df.pinyin.iloc[j]).split(), path_num = 1)
    for i,item in enumerate(dagg):
      df.P2H_DAG_gen.iloc[j] = ''.join(re.findall(pre, str(item.path)))
    t1_stop=time.time()
    df.P2H_DAG_time.iloc[j] = t1_stop - t1_start
    print("No",j,"DAG_gen",df.P2H_DAG_gen.iloc[j])
    print("No",j,"DAG_time",t1_stop - t1_start)


No 0 DAG_gen 笼子
No 0 DAG_time 0.000835418701171875
No 1 DAG_gen 完白
No 1 DAG_time 0.0004544258117675781
No 2 DAG_gen 诚邀
No 2 DAG_time 0.0004894733428955078
No 3 DAG_gen 辞去
No 3 DAG_time 0.0003657341003417969
No 4 DAG_gen 漂流
No 4 DAG_time 0.0003802776336669922
No 5 DAG_gen 枪手
No 5 DAG_time 0.0005044937133789062
No 6 DAG_gen 驾校
No 6 DAG_time 0.0004057884216308594
No 7 DAG_gen 神教
No 7 DAG_time 0.0004248619079589844
No 8 DAG_gen 说唱
No 8 DAG_time 0.0003712177276611328
No 9 DAG_gen 简评
No 9 DAG_time 0.0003509521484375
No 10 DAG_gen 干将
No 10 DAG_time 0.0003352165222167969
No 11 DAG_gen 巡查
No 11 DAG_time 0.00035119056701660156
No 12 DAG_gen 强忍
No 12 DAG_time 0.000362396240234375
No 13 DAG_gen 基金
No 13 DAG_time 0.00034236907958984375
No 14 DAG_gen 管制
No 14 DAG_time 0.0003509521484375
No 15 DAG_gen 小说
No 15 DAG_time 0.0005524158477783203
No 16 DAG_gen 但要
No 16 DAG_time 0.0004048347473144531
No 17 DAG_gen 户责
No 17 DAG_time 0.0004029273986816406
No 18 DAG_gen 海鲜
No 18 DAG_time 0.0003960132598876953


In [ ]:
'''
df=Chinese sentence
'''
dagparams = DefaultDagParams()
pre = re.compile(u'[\u4e00-\u9fa5-\，\。]')

for j in range(df.shape[0]):
  text=re.sub(r'\d+','',simplify_pinyin(df.pinyin_nopun.iloc[j]))

  t1_start=time.time()
  dagg = dag(dagparams, text.split(), path_num = 1)
  for i,item in enumerate(dagg):
      df.P2H_HMM_gen.iloc[j] = ''.join(re.findall(pre, str(item.path)))
  t1_stop=time.time()

## Gemini

Used for both Gemini1.5 Flash and Gemini2.0 Flash

### Chinese Word Generation

In [ ]:
f"""
Convert this pinyin: "{pinyin}" to the corresponding Chinese characters. Return just the characters.
"""

### Chinese Sentence Generation

In [ ]:
f"""
Convert the provided pinyin to {chn_type} Chinese, return your results only, do not explain: {pinyin}
"""

### Japanese Word Generation

In [ ]:
f"""
Convert this Romaji: "{romanji}" to Japanese {to}. Return just the Japanese {to}.
"""

### Japanese Sentence Generation

In [ ]:
f"""
Convert the Romaji sentence: "{romanji}" to Japanese. Return just the Japanese sentence.
"""

### Chinese Error Correction

Pinyin Correction

In [ ]:
f"""
Correct and convert this pinyin sentence: "{pinyin}" to Chinese characters. Only return the Chinese sentence.
"""

Chinese Characters Correction

In [ ]:
f"""
Correct any errors in this Chinese sentence: "{incorrectstring}". Only return the corrected Chinese sentence.
"""

### Japanese Error Correction

Romaji Correction

In [ ]:
f"""
Correct and convert this Romaji sentence: "{pinyin}" to Japanese characters. Only return the Japanese sentence.
"""

Japanese Characters Correction

In [ ]:
f"""
Correct any errors in this Japanese sentence: "{incorrectstring}". Only return the corrected Japanese sentence.
"""

## Llama

Used for both Llama3-8B and Llama3.2-3B


### Chinese Word Generation

In [ ]:
f"""
Convert the following Chinese pinyin word: "{pinyin}" to the corresponding Chinese characters.
Only respond with the Chinese characters.

Example:
user: ni hao
output: 你好
"""

### Chinese Sentence Generation

In [ ]:
f"""
Convert the following Chinese pinyin sentence: "{pinyin}" to {categ} Chinese.
Only respond with the {categ} Chinese sentence.
"""

### Japanese Word Generation

In [ ]:
f"""
Convert the following Japanese romaji word: "{romaji}" to the corresponding Japanese {categ} characters.
Only respond with the Japanese characters.
"""

### Japanese Sentence Generation

In [ ]:
f"""
Convert the following Japanese romaji sentence: "{romaji}" to Japanese.
Only respond with the Japanese sentence.
"""

### Chinese Error Correction

Pinyin Correction

In [ ]:
f"""
Correct errors and convert the following Chinese pinyin sentence: "{pinyin}" to Chinese.
Only respond with the Chinese sentence.
"""

Chinese Characters Correction

In [ ]:
f"""
Correct errors in the following Chinese sentence: "{pinyin}".
Only respond with the Chinese sentence.
"""

### Japanese Error Correction

Romaji Correction

In [ ]:
f"""
Correct errors and convert the following Japanese romaji sentence: "{romaji}" to Japanese.
Only respond with the Japanese sentence.
"""

Japanese Characters Correction

In [ ]:
f"""
Correct errors in the following Japanese sentence: "{romaji}".
Only respond with the Japanese sentence.
"""

## Baichuan

### Chinese Word Generation

In [ ]:
f"""
你会从用户输入拼音，发出相应的简体中文。

```json
{{
  "original_string": string  // This is a user input Pinyin string
  "ime_string": string  // This is your first response, the Chinese response
}}```

% 用户输入：
{user_input}

% 你的响应：
"""

### Chinese Sentence Generation

In [ ]:
f"""
你会从用户输入拼音，发出相应的{chn_type}中文。

```json
{{
  "original_string": string  // This is a user input Pinyin string
  "ime_string": string  // This is your first response, the Chinese response
}}```

% 用户输入：
{user_input}

% 你的响应：
"""

### Chinese Error Correction

Pinyin Correction

In [ ]:
"""
帮用户把可能有错误的中文拼音修正，然后翻译成汉字。你一定要只将修正的句子回答应。
"""

Chinese Character Correction

In [ ]:
"""
帮用户把可能有错误的中文汉字修正。你一定要只将修正的句子回答应。
"""

## GLM4

### Chinese Word Generation

In [ ]:
f"""
You will be given a Pinyin from a user, try to output the corresponding Simplified Chinese.

The output should be a markdown code snippet formatted in the following schema,
including the leading and trailing "```json" and "```":

```json
{{
  "original_string": string  // This is a user input Pinyin string
  "ime_string": string  // This is your first response, the Chinese response
}}```

% USER INPUT:
{user_input}

% YOUR RESPONSE:
"""

### Chinese Sentence Generation

In [ ]:
f"""
You will be given a Pinyin from a user, try to output the corresponding {chn_type} Chinese.

The output should be a markdown code snippet formatted in the following schema,
including the leading and trailing "```json" and "```":

```json
{{
  "original_string": string  // This is a user input Pinyin string
  "ime_string": string  // This is your first response, the Chinese response
}}```

% USER INPUT:
{user_input}

% YOUR RESPONSE:
"""

### Japanese Word Generation

In [ ]:
f"""
You will be given a romaji from a user, try to output the corresponding Japanese.

The output should be a markdown code snippet formatted in the following schema,
including the leading and trailing "```json" and "```":

```json
{{
  "original_string": string  // This is a user input romaji string
  "ime_string": string  // This is your first response, the Japanese response
}}```

% USER INPUT:
{user_input}

% YOUR RESPONSE:
"""

### Japanese Sentence Generation

In [ ]:
f"""
You will be given a romaji from a user, try to output the corresponding Japanese.

The output should be a markdown code snippet formatted in the following schema,
including the leading and trailing "```json" and "```":

```json
{{
  "original_string": string  // This is a user input romaji string
  "ime_string": string  // This is your first response, the Japanese response
}}```

% USER INPUT:
{user_input}

% YOUR RESPONSE:
"""

### Chinese Error Correction

Pinyin Correction

In [ ]:
f"""
You will be given a string of Chinese pinyin,  which may contain slight errors.
This includes various kinds of typos including missing letters, extra letters,
and keyboard mispresses according to nearby letter keys, etc.

Typo Examples:

Input: "ni hai ma?"
Error: "hai" should be "hao"
Corrected Pinyin: "ni hao ma?"
Output Chinese: "你好吗?"

Input: "zhee shi wo de sh."
Error: "zhee" should be "zhe"; "sh" should "shu"
Corrected Pinyin: "zhe shi wo de shu."
Output Chinese: "这是我的书."

Input: "ta s wo de peng you."
Error: "s" should be "shi"
Corrected Pinyin: "ta shi wo de pengyou."
Output Chinese: "他是我的朋友."

Input: "qianmian yu yi jia diai."
Error: "yu" should be "you";"diai" should be "dian"
Corrected Pinyin: "qianmian you yi jia dian."
Output Chinese: "前面有一家店."

Try to correct the Pinyin and output the corresponding simplified Chinese.
Ensure the final output is a Chinese sentence, and directly comes from the pinyin.
"""

Chinese Characters Correction

In [ ]:
f"""
You will be given sentences with incorrect Chinese characters from a user,
which may contain some numbers, various kinds of errors including wrong characters
with typos, word misuse, grammatical errors, and more.

Error Examples:

Input: "白血斌"
Error: "斌" should be "病"
Corrected Chinese: "白血病"

Input: "我要去北京tiananman广场玩儿。"
Error: "tiananman" should be "天安门"
Corrected Chinese: "我要去北京天安门广场玩儿。"

Input: "金秋9月，给大高校开学的钟声陆续敲响，全国各地先去了周边短租热潮"
Error: "给大" should be "各大"; "先去" should "掀起"
Corrected Chinese: "金秋9月，各大高校开学的钟声陆续敲响，全国各地掀起了周边短租热潮"


Input: "意味着这些第3方应用灭下来将无法向用户发送私"
Error: "灭下来" should be “接下来"; "私" should be "私信"
Corrected Chinese: "意味着这些第3方应用接下来将无法向用户发送私信"


Try to correct the wrong Chinese characters and output the corresponding correct simplified Chinese sentence.
Ensure the final output is a Chinese sentence.
"""

### Japanese Error Correction

Romaji Correction

In [ ]:
f"""
You will be given strings with incorrect Romanji from a user, which may contain
various kinds of typos including missing or adding some letters, mispress or
substitute other characters, and grammatical errors and more.

Try to correct the Romanji and output the corresponding Japanese sentence.
"""

Japanese Characters Correction

In [ ]:
f"""
You will be given sentences with incorrect Japanese characters from a user, which
may contain various kinds of errors including replacement of a character with an
erroneous one,the omission of a necessary character, the addition of an
unnecessary character, and character misuse, grammatical errors, and more.

Try to correct the wrong Japanses characters and output the corresponding correct Japanese sentence.
"""

## Rinna

### Japanese Word Generation

In [ ]:
f"""
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字の入力を、日本語の文字に正確に変換してください。

### 指示:
ローマ字を日本語の文字に変換してください。可能な限り正確に翻訳してください。

### 入力:
{input}

### 応答:
"""

### Japanese Sentence Generation

In [ ]:
f"""
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。ローマ字の入力を、日本語の文字に正確に変換してください。

### 指示:
ユーザーからローマ字を与えられたら、対応する日本語を出力してみます。

### 入力:
{input}

### 応答:
"""

### Japanese Error Correction

Romaji Correction

In [ ]:
f"""
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
ローマ字を修正して日本語訳を返してください。

### 入力:
{romanji}

### 応答:
"""

Japanese Character Correction

In [ ]:
f"""
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示:
入力された文章を修正し、日本語の文章を返します

### 入力:
{input}

### 応答:
"""